## Análisis de datos - Clase 5

### Técnicas avanzadas para el tratamiento de outliers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer

In [2]:
# Cargar el dataset de Titanic
df_titanic = sns.load_dataset("titanic")

# Seleccionar variables numéricas relevantes
df = df_titanic[['age', 'fare']].dropna()
df.head(10)

,age,fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500
6,54.0,51.8625
7,2.0,21.0750
8,27.0,11.1333
9,14.0,30.0708
10,4.0,16.7000


In [3]:
df.describe()

,age,fare
count,714.000000,714.000000
mean,29.699118,34.694514
std,14.526497,52.918930
min,0.420000,0.000000
25%,20.125000,8.050000
50%,28.000000,15.741700
75%,38.000000,33.375000
max,80.000000,512.329200


### Detectamos outliers en base al rango intercuartil

Un dato se considera outlier si es < (Q1 - 1.5 * IQR)) o > (Q3 + 1.5 * IQR)

In [4]:
# Métodos estadísticos para detectar outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)

IQR = Q3 - Q1
outliers_iqr = (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))
print(f"Outliers\nAge: <{Q1['age'] - 1.5 * IQR['age']:.02f} o >{Q3['age'] + 1.5 * IQR['age']:.02f}")
print(f"Fare: <{Q1['fare'] - 1.5 * IQR['fare']:.02f} o >{Q3['fare'] + 1.5 * IQR['fare']:.02f}")

Outliers
Age: <-6.69 o >64.81
Fare: <-29.94 o >71.36


### Estrategias de eliminación y tratamiento de outliers

#### 3. Imputación

#### Imputación con KNN

In [5]:
imputer = KNNImputer(n_neighbors=5)
df_nan = df.copy()

df_nan[outliers_iqr] = np.nan  # marca outliers como NaN

df_knn_imputed = pd.DataFrame(imputer.fit_transform(df_nan),
                              columns=df.columns,
                              index=df_nan.index # sin esto, se pierde el índice original
                             )

df_knn_imputed.describe()

,age,fare
count,714.000000,714.000000
mean,29.168305,19.606931
std,13.683456,13.939995
min,0.420000,0.000000
25%,20.125000,8.050000
50%,28.000000,14.454200
75%,38.000000,26.287500
max,64.000000,71.283300


In [6]:
# comparo las observaciones originales con las imputaciones
df_compara_knn = pd.concat([df, df_nan, df_knn_imputed], axis=1)

df_compara_knn.columns = ['age_original', 'fare_original', 'age_nan', 'fare_nan', 'age_imputado', 'fare_imputado']

df_compara_knn.head(30) # se observan imputaciones de Fare

,age_original,fare_original,age_nan,fare_nan,age_imputado,fare_imputado
0,22.0,7.2500,22.0,7.2500,22.0,7.25000
1,38.0,71.2833,38.0,71.2833,38.0,71.28330
2,26.0,7.9250,26.0,7.9250,26.0,7.92500
3,35.0,53.1000,35.0,53.1000,35.0,53.10000
4,35.0,8.0500,35.0,8.0500,35.0,8.05000
6,54.0,51.8625,54.0,51.8625,54.0,51.86250
7,2.0,21.0750,2.0,21.0750,2.0,21.07500
8,27.0,11.1333,27.0,11.1333,27.0,11.13330
9,14.0,30.0708,14.0,30.0708,14.0,30.07080
10,4.0,16.7000,4.0,16.7000,4.0,16.70000


In [7]:
df_compara_knn.sort_values('age_original')  # para ver las imputaciones de Age

,age_original,fare_original,age_nan,fare_nan,age_imputado,fare_imputado
803,0.42,8.5167,0.42,8.5167,0.42,8.5167
755,0.67,14.5000,0.67,14.5000,0.67,14.5000
644,0.75,19.2583,0.75,19.2583,0.75,19.2583
469,0.75,19.2583,0.75,19.2583,0.75,19.2583
78,0.83,29.0000,0.83,29.0000,0.83,29.0000
...,...,...,...,...,...,...
116,70.50,7.7500,NaN,7.7500,32.20,7.7500
493,71.00,49.5042,NaN,49.5042,36.80,49.5042
96,71.00,34.6542,NaN,34.6542,28.20,34.6542
851,74.00,7.7750,NaN,7.7750,24.60,7.7750
